# Analysis of the autism data

In [30]:
run init.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done.


# Experiments

* This section aims at answering a few questions related to the different algorithms used, with the aim of optimizing our classification framework.

**Parameters or potential settings**

* The approach used, which could be Logistic Regression, Decision Tree, Explainable Boosting Machine, Neural Additive models, or XGBoost.
* The use or not of indicator variables as inputs. 
* The scaling of the data. 
* The imputation approach when the learning algorithms does not handle them by design. It could be constant imputation (called encoding here), or conditional imputation (mean, knn, or mice).
* The sampling method in the case of imbalance learning (either no down-sampling, vanilla (random sampling of the minority class until classes are balanced), or SMOTE (a more elaborated sampling).
* The number of features, between 2 and 6. 
* The number of folds when cross-validating the results. 

**Notes on the classification pipeline:**

* Two datasets are age-matched but one (scenario `asd_td_age_matched_n_balanced`) lead to relatively balanced classes, whereas the scenario `asd_td_age_matched_n_unbalanced` has slightly younger kids and so leverage the amount of young neuro-typical kids, and is more unbalanced. 
* No searches on hyper-parameters are performed on each of the approach. Cross-validation is performed with the stratified inner-fold being left-out, the model is fitted on the training set, and the prediction of the test set is done with predictions stored for later evaluation of performances. 
* since a lot of settings parameters are tested, we test all of the hypothesis with a default setting that is: Encoding of the missing variables, scaling of the data, no use of indicator variables, no down-sampling (???), and a 16-fold cross-validation.
* Features selection were made for the two scenario suing the features with the higher importance based on xgboost importance map.
* Classification here is between autistic and neurotypical participants.



**Among the questions at stakes:**

* Experiment 1: Shall we use indicator variables? For each scenario (columns of axes), ad each dimension of the problem (rows of axes) , x is the `use_of_indicator_variables`, y is an indicator of performances (typically the F1 score), and the hue variable are the approaches. The plots are performed for no imputations, 
* Experiement 2: Shall we scale data or not ? 
* Experiment X: for the algorithms handling missing variables (xgboost, nam with encoding) , shall we let them be missing, or imputed ? 

In [31]:
df =  pd.read_csv(AUTISM_DATA_PATH)
scenario='papers'

with open(os.path.join(DATA_DIR, 'selected_features_{}.pkl'.format(scenario)), 'rb') as f:
    features = pickle.load(f)

In [32]:
list(features['xgboost'].keys())

[10, 22, 32]

## Experiment 1.  `use_missing_indicator_variables`

In [34]:
n=0
for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for use_missing_indicator_variables in [True, False]:
            
            for approach in ['nam', "LogisticRegression", "DecisionTree",  "ebm", 'xgboost']:
                
                df =  pd.read_csv(AUTISM_DATA_PATH)


                data = Dataset(df=df, 
                               missing_data_handling='encoding',
                               imputation_method='without',
                               sampling_method='without',
                               scenario = scenario, 
                               features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                               scale_data=True, 
                               use_missing_indicator_variables=use_missing_indicator_variables,
                               verbosity=0, 
                               proportion_train=1 if approach != 'nam' else .8)

                exp = Experiments(data.dataset_name,
                                  dataset=data, 
                                  approach=approach, 
                                  previous_experiment=None,        
                                  debug=False, 
                                  verbosity=0, 
                                  experiment_folder_name='paper_experiment_1_fs_validation',
                                  save_experiment=True)
                
                
                print("approach: {} - use_missing_indicator_variables: {} - n_features: {}".format(approach, use_missing_indicator_variables, n_features))
                exp.fit()
                
                exp.dataset.proportion_train = 0.
                exp.dataset._init_scenario(scenario='papers_remote')
                exp.predict()
                
                display(exp.performances_df)
                

approach: nam - use_missing_indicator_variables: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV),Area Under the Curve (AUC)
0,0.667,0.569,0.298,0.061,0.18,0.677,0.104,0.58,0.339,0.977,0.281,0.506,0.941,0.023,0.494,0.059,0.667


approach: LogisticRegression - use_missing_indicator_variables: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.639,0.554,0.292,0.1,0.769,0.65,0.13,0.57,0.252,0.884,0.333,0.5,0.792,0.116,0.5,0.208


approach: DecisionTree - use_missing_indicator_variables: True - n_features: 10


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.533,0.453,0.058,0.026,0.452,0.601,0.058,0.58,0.091,0.209,0.86,0.529,0.59,0.791,0.471,0.41


approach: ebm - use_missing_indicator_variables: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.679,0.608,0.323,0.062,0.577,0.678,0.234,0.61,0.331,0.907,0.386,0.527,0.846,0.093,0.473,0.154


approach: xgboost - use_missing_indicator_variables: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.681,0.611,0.36,0.111,0.876,0.656,0.13,0.56,0.266,0.93,0.281,0.494,0.842,0.07,0.506,0.158


## Experiment 2  `scale_data`

In [6]:

for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for scale_data in [True, False]:
            
            for approach in ["LogisticRegression", "DecisionTree", "nam", "ebm", 'xgboost']:
                
                df =  pd.read_csv(AUTISM_DATA_PATH)


                data = Dataset(df=df, 
                               missing_data_handling='encoding',
                               imputation_method='without',
                               sampling_method='without',
                               scenario = scenario, 
                               features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                               scale_data=scale_data, 
                               use_missing_indicator_variables=False,
                               verbosity=1, 
                               proportion_train=1 if approach != 'nam' else .8)

                exp = Experiments(data.dataset_name,
                                  dataset=data, 
                                  approach=approach, 
                                  previous_experiment=None,        
                                  debug=False, 
                                  verbosity=0, 
                                experiment_folder_name='paper_experiment_2_fs_validation',
                                  save_experiment=True)

                print("approach: {} - scale_data: {} - n_features: {}".format(approach, scale_data, n_features))
                exp.fit()

                exp.dataset.proportion_train = 0.
                exp.dataset._init_scenario(scenario='papers_remote')
                exp.predict()

                display(exp.performances_df)                    


approach: LogisticRegression - scale_data: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.639,0.547,0.292,0.078,0.678,0.65,0.089,0.57,0.252,0.884,0.333,0.5,0.792,0.116,0.5,0.208


approach: DecisionTree - scale_data: True - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.558,0.5,0.085,0.097,0.904,0.601,0.13,0.42,-0.116,0.977,0.0,0.424,0.0,0.023,0.576,1.0


approach: nam - scale_data: True - n_features: 10


...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV),Area Under the Curve (AUC)
0,0.605,0.531,0.237,0.036,NaN,0.628,0.108,0.48,0.159,0.977,0.105,0.452,0.857,0.023,0.548,0.143,0.605


approach: ebm - scale_data: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.698,0.646,0.424,0.178,0.938,0.661,0.245,0.65,0.309,0.698,0.614,0.577,0.729,0.302,0.423,0.271


approach: xgboost - scale_data: False - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.689,0.587,0.39,0.045,NaN,0.667,0.126,0.61,0.304,0.86,0.421,0.529,0.8,0.14,0.471,0.2


## Experiment 3. `Imputation approach`

In [27]:

for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for imputation_method in ['mice', 'knn', 'mean']:
            
            for approach in ["LogisticRegression", "DecisionTree", "nam", "ebm", 'xgboost']:
                
                df =  pd.read_csv(AUTISM_DATA_PATH)

                data = Dataset(df=df, 
                               missing_data_handling='imputation',
                               imputation_method=imputation_method,
                               sampling_method='without',
                               scenario = scenario, 
                               features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                               scale_data=True, 
                               use_missing_indicator_variables=False,
                               verbosity=0, 
                               proportion_train=1 if approach != 'nam' else .8)

                exp = Experiments(data.dataset_name,
                                  dataset=data, 
                                  approach=approach, 
                                  previous_experiment=None,        
                                  debug=False, 
                                 experiment_folder_name='paper_experiment_3_fs_validation',
                                  verbosity=0, 
                                  save_experiment=True)

                print("approach: {} - imputation_method: {} - n_features: {}".format(approach, imputation_method, n_features))
                exp.fit()

                exp.dataset.proportion_train = 0
                exp.dataset._init_scenario(scenario='papers_remote')
                exp.predict()

                display(exp.performances_df)                     


approach: LogisticRegression - imputation_method: mice - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.684,0.621,0.382,0.06,0.581,0.655,0.17,0.58,0.268,0.884,0.351,0.507,0.8,0.116,0.493,0.2


approach: DecisionTree - imputation_method: mice - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.656,0.591,0.333,0.172,0.955,0.601,0.245,0.62,0.245,0.651,0.596,0.549,0.694,0.349,0.451,0.306


approach: nam - imputation_method: mice - n_features: 10
	===== Replicate no. 1 =====

	===== Replicate no. 2 =====

	===== Replicate no. 3 =====

	===== Replicate no. 4 =====

	===== Replicate no. 5 =====

	===== Replicate no. 6 =====

	===== Replicate no. 7 =====

	===== Replicate no. 8 =====

	===== Replicate no. 9 =====

	===== Replicate no. 10 =====



...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV),Area Under the Curve (AUC)
0,0.656,0.601,0.265,0.133,0.918,0.683,0.17,0.59,0.354,0.977,0.298,0.512,0.944,0.023,0.488,0.056,0.656


approach: ebm - imputation_method: mean - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.702,0.665,0.386,0.174,0.957,0.678,0.234,0.61,0.331,0.907,0.386,0.527,0.846,0.093,0.473,0.154


approach: xgboost - imputation_method: mean - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.669,0.633,0.33,0.143,0.939,0.655,0.192,0.6,0.276,0.837,0.421,0.522,0.774,0.163,0.478,0.226


## Experiment 4. `Sampling approach`

In [4]:
n=0
for i, n_features in enumerate(list(features['xgboost'].keys())):
    for j, scenario in enumerate(['papers']):
        
        for sampling_method in ['without', 'vanilla', 'smote']:
            
            for approach in ["LogisticRegression", "DecisionTree", "nam", "ebm", 'xgboost']:
                    
                df =  pd.read_csv(AUTISM_DATA_PATH)


                data = Dataset(df=df, 
                               missing_data_handling='encoding',
                               imputation_method='without',
                               sampling_method=sampling_method,
                               scenario = scenario, 
                                   features_name = features[approach][n_features] if approach in features.keys() else features['ebm'][n_features],
                               scale_data=True, 
                               use_missing_indicator_variables=False,
                               verbosity=1, 
                               proportion_train=1 if approach != 'nam' else .8)

                exp = Experiments(data.dataset_name,
                                  dataset=data, 
                                  approach=approach, 
                                  previous_experiment=None,        
                                  debug=False, 
                                 experiment_folder_name='paper_experiment_4_fs_validation',
                                  verbosity=0, 
                                  save_experiment=True)
                print("approach: {} - sampling_method: {} - n_features: {}".format(approach, sampling_method, n_features))
                
                exp.fit()
                
                exp.dataset.proportion_train = 0.
                exp.dataset._init_scenario(scenario='papers_remote')
                exp.predict()
                
                
                display(exp.performances_df)  
       

approach: LogisticRegression - sampling_method: without - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.639,0.547,0.292,0.078,0.678,0.65,0.089,0.57,0.252,0.884,0.333,0.5,0.792,0.116,0.5,0.208


approach: DecisionTree - sampling_method: without - n_features: 10


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.558,0.5,0.085,0.097,0.904,0.601,0.13,0.42,-0.116,0.977,0.0,0.424,0.0,0.023,0.576,1.0


approach: nam - sampling_method: without - n_features: 10
	===== Replicate no. 1 =====

	===== Replicate no. 2 =====

	===== Replicate no. 3 =====

	===== Replicate no. 4 =====

	===== Replicate no. 5 =====

	===== Replicate no. 6 =====

	===== Replicate no. 7 =====

	===== Replicate no. 8 =====

	===== Replicate no. 9 =====

	===== Replicate no. 10 =====



...

,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV),Area Under the Curve (AUC)
0,0.671,0.616,0.273,0.134,0.915,0.678,0.17,0.62,0.333,0.884,0.421,0.535,0.828,0.116,0.465,0.172,0.671


approach: ebm - sampling_method: smote - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.679,0.608,0.323,0.062,0.577,0.678,0.234,0.61,0.331,0.907,0.386,0.527,0.846,0.093,0.473,0.154


approach: xgboost - sampling_method: smote - n_features: 32


,AUROC,AUC-PR,AUC-PR-Gain,AUC-PR-Corrected,AUC-PR-Gain-Corrected,F1 score (2 PPVxTPR/(PPV+TPR)),F1 score Corrected,Accuracy,Matthews correlation coefficient (MCC),"Sensitivity, recall, hit rate, or true positive rate (TPR)","Specificity, selectivity or true negative rate (TNR)",Precision or positive predictive value (PPV),Negative predictive value (NPV),Miss rate or false negative rate (FNR),False discovery rate (FDR=1-PPV),False omission rate (FOR=1-NPV)
0,0.681,0.611,0.36,0.111,0.876,0.656,0.13,0.56,0.266,0.93,0.281,0.494,0.842,0.07,0.506,0.158
